In [ ]:
# 필요한 라이브러리 설치
!pip install pandas langchain langchain-community langchain-chroma sentence-transformers torch

import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import os
from pprint import pprint

# -----------------------------------------------------------------------------
# 1. 데이터 로드 및 전처리
# -----------------------------------------------------------------------------
# 사용자가 제공한 'titanic.csv' 파일을 불러옵니다.
csv_file_path = './data/titanic.csv'
df = pd.read_csv(csv_file_path)

# 챗봇이 더 풍부한 정보를 제공할 수 있도록 여러 컬럼을 결합하여 새로운 컬럼을 생성합니다.
# 'Name', 'Sex', 'Age', 'Pclass', 'Survived' 정보를 결합하여 'combined_info' 컬럼을 만듭니다.
# Survived 컬럼의 0과 1을 '사망'과 '생존'으로 변환하여 가독성을 높입니다.
df['Survived_str'] = df['Survived'].apply(lambda x: '생존' if x == 1 else '사망')

df['combined_info'] = (
    df['Name'] + "은(는) " +
    df['Sex'] + "성 승객으로, 나이는 " + df['Age'].astype(str) + "세입니다. " +
    "탑승 등급은 " + df['Pclass'].astype(str) + "등급이었으며, 최종적으로 " +
    df['Survived_str'] + "했습니다."
)

# LangChain의 DataFrameLoader를 사용하여 DataFrame을 'Document' 객체로 변환합니다.
# 'page_content_column'을 새로 만든 'combined_info' 컬럼으로 지정합니다.
loader = DataFrameLoader(df, page_content_column='combined_info')
docs = loader.load()

# 문서가 너무 길 경우, 검색 정확도를 높이기 위해 작은 단위로 나눕니다.
# RecursiveCharacterTextSplitter는 다양한 구분자를 활용하여 문서를 나눕니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # 한 덩어리의 최대 크기
    chunk_overlap=50,      # 덩어리 간의 중복 크기
    length_function=len
)
split_docs = text_splitter.split_documents(docs)

print(f"원본 문서 수: {len(docs)}")
print(f"분할된 문서 수: {len(split_docs)}\n")

In [ ]:

# -----------------------------------------------------------------------------
# 2. 임베딩 모델 및 벡터 스토어 설정
# -----------------------------------------------------------------------------
# BGE 한글 임베딩 모델을 불러옵니다.
# 'bge-large-ko'는 한글에 최적화된 모델 중 하나입니다.
# model_kwargs는 GPU를 사용할 경우 'cuda'로 설정할 수 있습니다.
model_name = "BAAI/bge-large-ko"
model_kwargs = {'device': 'cpu'}  # GPU 사용 시 'cuda'로 변경
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# ChromaDB를 사용하여 임베딩된 문서를 저장합니다.
# 'persist_directory'를 지정하면 데이터가 디스크에 저장되어 다음에 재활용할 수 있습니다.
persist_directory = 'chroma_db'
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

# 쿼리에 가장 유사한 문서를 찾아주는 'retriever'를 생성합니다.
# 'k=3'은 가장 유사한 3개의 문서를 가져오겠다는 의미입니다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})



In [ ]:
# -----------------------------------------------------------------------------
# 3. LLM 모델 및 RAG Chain 구성
# -----------------------------------------------------------------------------
# Ollama를 사용하여 로컬에서 실행 중인 LLM을 불러옵니다.
# 'llama3' 대신 'beomi/KoAlpaca-Polyglot-5.8B'와 같은 한글 모델을 사용할 수 있습니다.
# https://ollama.com/library/llama3
# https://ollama.com/library/koalpaca
# ollama pull llama3 또는 ollama pull koalpaca-polyglot:latest
#
# 만약 Ollama 설치가 어렵다면, 주석 처리하고 다른 LLM API를 사용하거나,
# HuggingFaceHub를 사용해 API 호출을 할 수도 있습니다.
# from langchain_community.llms import HuggingFaceHub
# llm = HuggingFaceHub(repo_id="beomi/KoAlpaca-Polyglot-5.8B", ...)

llm = ChatOllama(model="llama3")

# RAG 시스템에 사용될 프롬프트 템플릿을 정의합니다.
# 'context'와 'question' 변수를 사용하여 LLM에게 배경 정보와 질문을 함께 전달합니다.
template = """
당신은 사용자의 질문에 답하는 친절한 한국어 챗봇입니다.
주어진 문맥(context)을 사용하여 질문(question)에 답하세요.
만약 문맥에 관련 정보가 없거나 질문에 답할 수 없다면, 모른다고 답변하세요.

문맥: {context}

질문: {question}

답변:
"""
prompt = PromptTemplate.from_template(template)

# RAG Chain을 구성합니다.
# 1. RunnablePassthrough: 사용자의 질문을 retriever와 prompt로 전달합니다.
# 2. retriever: 질문에 맞는 문서를 검색합니다.
# 3. prompt: 검색된 문서를 context로, 질문을 question으로 넣어 프롬프트를 만듭니다.
# 4. llm: 완성된 프롬프트를 기반으로 답변을 생성합니다.
# 5. StrOutputParser: LLM의 응답을 문자열로 파싱합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("챗봇이 준비되었습니다. '종료'를 입력하면 대화가 끝납니다.\n")

In [ ]:
# -----------------------------------------------------------------------------
# 4. 챗봇 대화 루프
# -----------------------------------------------------------------------------
while True:
    try:
        user_question = input("질문하세요: ")
        if user_question.lower() == '종료':
            print("대화를 종료합니다.")
            break

        # RAG Chain을 실행하여 답변을 얻습니다.
        result = rag_chain.invoke(user_question)
        print("챗봇 답변:")
        pprint(result)

    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        break

# 사용이 끝난 벡터 스토어 데이터 삭제 (선택 사항)
# import shutil
# if os.path.exists(persist_directory):
#     shutil.rmtree(persist_directory)
#     print(f"'{persist_directory}' 디렉토리를 삭제했습니다.")